# Tratamento e Limpeza de Dados
## Classificação de Intensidade de Chuva - Problema 7.8

**Objetivo:** Tratar valores faltantes, limpar dados e classificar a intensidade de chuva.

**Classificação:**
- Sem chuva: 0 mm
- Leve: 0.1 - 2.5 mm/h
- Moderada: 2.6 - 10 mm/h
- Forte: > 10 mm/h

---


## 1. Importação de Bibliotecas


In [ ]:
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package.split('-')[0].split('[')[0])
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", package])

dependencies = ['pandas', 'numpy', 'psycopg2-binary']

for dep in dependencies:
    install_if_missing(dep)

import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_batch
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")


✅ Bibliotecas importadas com sucesso!


## 2. Conexão e Carregamento


In [ ]:
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'inmet_db',
    'user': 'inmet_user',
    'password': 'inmet_password'
}

query = "SELECT * FROM dados_meteorologicos ORDER BY timestamp_utc"

try:
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_client_encoding('UTF8')
    df = pd.read_sql(query, conn)
    conn.close()
    df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
    print(f"Dados carregados: {len(df):,} registros")
except Exception as e:
    print(f"Erro: {e}")
    df = pd.DataFrame()

if len(df) > 0:
    print(f"Valores faltantes: {df.isnull().sum().sum()}")
    display(df.head())


✅ Dados carregados: 96,480 registros
Valores faltantes: 497162


,id,codigo_wmo,data,hora_utc,timestamp_utc,precipitacao_mm,pressao_estacao_mb,pressao_max_mb,pressao_min_mb,radiacao_global_kjm2,...,temperatura_orvalho_max_c,temperatura_orvalho_min_c,umidade_rel_max_pct,umidade_rel_min_pct,umidade_rel_horaria_pct,vento_direcao_graus,vento_rajada_max_ms,vento_velocidade_ms,intensidade_chuva,created_at
0,8785,A366,2024-01-01,00:00:00,2024-01-01,0.0,960.8,960.8,960.4,NaN,...,20.1,19.9,68.0,64.0,68.0,253.0,7.0,1.9,sem_chuva,2025-11-30 02:20:59.997127
1,1,A357,2024-01-01,00:00:00,2024-01-01,0.0,993.9,993.9,993.3,NaN,...,23.0,22.8,86.0,84.0,86.0,32.0,2.8,1.2,sem_chuva,2025-11-30 02:20:53.993287
2,48241,A322,2024-01-01,00:00:00,2024-01-01,NaN,922.1,922.1,921.4,NaN,...,NaN,NaN,NaN,NaN,NaN,64.0,6.8,3.5,sem_chuva,2025-11-30 02:21:46.131988
3,57025,A350,2024-01-01,00:00:00,2024-01-01,0.8,956.2,956.5,955.9,39.8,...,20.6,19.1,89.0,74.0,86.0,64.0,4.9,1.1,leve,2025-11-30 02:22:03.241770
4,17569,A328,2024-01-01,00:00:00,2024-01-01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sem_chuva,2025-11-30 02:21:05.824536


## 3. Tratamento de Valores Faltantes


In [15]:
if len(df) > 0:
    # Precipitação nula = 0 (sem chuva)
    df['precipitacao_mm'] = df['precipitacao_mm'].fillna(0)
    
    # Outras variáveis: mediana por estação
    vars_numericas = ['pressao_estacao_mb', 'temperatura_ar_c', 'umidade_rel_horaria_pct', 
                      'vento_velocidade_ms', 'radiacao_global_kjm2']
    
    for var in vars_numericas:
        if var in df.columns:
            medianas = df.groupby('codigo_wmo')[var].transform('median')
            df[var] = df[var].fillna(medianas).fillna(df[var].median())
    
    print(f"✅ Valores faltantes após tratamento: {df.isnull().sum().sum()}")
else:
    print("⚠️  Nenhum dado disponível!")


✅ Valores faltantes após tratamento: 300173


## 4. Classificação de Intensidade de Chuva


In [16]:
def classificar_intensidade_chuva(precipitacao):
    """Classifica conforme problema 7.8"""
    if pd.isna(precipitacao) or precipitacao == 0:
        return 'sem_chuva'
    elif 0.1 <= precipitacao <= 2.5:
        return 'leve'
    elif 2.6 <= precipitacao <= 10:
        return 'moderada'
    elif precipitacao > 10:
        return 'forte'
    else:
        return 'sem_chuva'

if len(df) > 0:
    df['intensidade_chuva'] = df['precipitacao_mm'].apply(classificar_intensidade_chuva)
    print("Distribuição de classes:")
    display(df['intensidade_chuva'].value_counts())


Distribuição de classes:


intensidade_chuva
sem_chuva    93133
leve          2855
moderada       428
forte           64
Name: count, dtype: int64

## 5. Atualização no Banco de Dados


In [ ]:
if len(df) > 0:
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        update_query = "UPDATE dados_meteorologicos SET intensidade_chuva = %s WHERE id = %s"
        dados_atualizar = [(row['intensidade_chuva'], row['id']) for _, row in df.iterrows()]
        execute_batch(cursor, update_query, dados_atualizar, page_size=1000)
        
        conn.commit()
        cursor.close()
        conn.close()
        
        print(f"{len(df):,} registros atualizados no banco!")
    except Exception as e:
        print(f"Erro: {e}")
else:
    print("Nenhum dado para atualizar!")


✅ 96,480 registros atualizados no banco!
